In [1]:
from solvs import Solver
from envs import MdpEnv
from gurobipy import Model, LinExpr, GRB
from time import thread_time_ns as thread_time

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2023-12-10
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


In [3]:
env = MdpEnv()
print(env.available_envs)

['Ex10', 'Ex20', 'Ex30', 'Ex31', 'Ex40', 'four_rooms', 'frozen_lake_eight', 'frozen_lake', 'taxi']


In [4]:
env_name = env.available_envs[4]
print(env_name)
env.make_env(env_name)

Ex40


In [7]:
class GurobiSolverPL:
    def __init__(self, env) -> None:
        self.env = env
        self.model = Model("MDP")
        self.model.setParam('OutputFlag', 0)
        self.model.setParam(GRB.Param.Threads, 1)
        self.model.setParam('LogToConsole', 0)
    
    def build(self):
        start_time = thread_time()
        self.var = []
        for _ in range(self.env.S):
            self.var.append(self.model.addVar(vtype=GRB.CONTINUOUS, lb=0))
        self.model.update()
        self.obj = LinExpr()
        for i in range(self.env.S):
            self.obj += self.var[i]/self.env.S
        self.model.setObjective(self.obj,GRB.MINIMIZE)
        for i in range(self.env.S):
            for j in range(self.env.A):
                total = 0
                for k in range(self.env.S):
                    total = total + self.env.gamma*self.env.P[j,i,k]*self.var[k]
                self.model.addConstr( self.var[i] >= self.env.R[i,j] + total, "Contrainte%d" % i)

        self.building_time = thread_time()-start_time

    def run(self):
        self.model.optimize()
        self.runtime = self.model.Runtime
        assert GRB.OPTIMAL == 2


In [9]:
solver = GurobiSolverPLDual(env)
solver.build()
solver.run()